In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import time
import os
from os import listdir
from os.path import isfile, join

import yfinance as yf

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# make plotly work in jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")

In [2]:
# Load the historic data downloaded

PATH = '/Users/iamsagar/Desktop/Trading Project/data/historical_all/'
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers.sort()

print(tickers)

['ABB_BO', 'AEGISLOG_BO', 'AMARAJABAT_BO', 'AMBALALSA_BO', 'ANDHRAPET_BO', 'ANSALAPI_BO', 'ARUNAHTEL_BO', 'ASIANHOTNR_BO', 'ATUL_BO', 'ATVPR_BO', 'BAJAJELEC_BO', 'BAJAJHIND_BO', 'BAJFINANCE_BO', 'BANCOINDIA_BO', 'BOMDYEING_BO', 'CENTURYTEX_BO', 'FORCEMOT_BO', 'HDFC_BO', 'UTIQUE_BO']


In [3]:
# Find daily returns and cumulative returns of stocks

def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1))-1
    return df

def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    return df

In [4]:
# Add Bollinger Bands

def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96*df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96*df['Close'].rolling(window=20).std() 
    return df


### Ichimoku Data to the Dataframe

- Conversion Line: Represents support, resistance and reversal. Used to measure short-term trends. 
- Baseline: Represents support, resistance and confirm trend changes. Called baseline because it lags price. 
- Leading Span A: Used to identify future areas of support and resistance. 
- Leading Span B: Another line used to identify support and resistance. 
- Lagging Span: Shows possible support and resistance. Used to confirm signals. 
- Cloud: Represents the divergence in price evolution. 

In [5]:
def add_ichimoku(df):
    # Conversion Line = (Highest value in period + Lowest value in period)/2 (9 sessions)
    hi_val = df['High'].rolling(window=9).max()
    lo_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + lo_val)/2
    
    # Baseline = (Highest value in period + Lowest value in period)/2 (26 sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    lo_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + lo_va)/2
    
    # Span A = (Conversion value + Base value)/2 (26 sessions)
    df['SpanA'] = ((df['Conversion'] + df['Baseline'])/2)
    
    # Span B = (Conversion value + Base value)/2 (52 sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    lo_val3 = df['Low'].rolling(window=52).min() 
    df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)
    
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)
    
    return df

In [6]:
def get_stock_df_from_csv(ticker):
    folder = '/Users/iamsagar/Desktop/Trading Project/data/historical_all/'
    try:
        df = pd.read_csv(folder + ticker + '.csv')
    except FileNotFoundError:
        print("File not found.")
    else:
        return df

In [10]:
# Checking on one stock if everything works

try:
    print("Working on :", "HDFC_BO")
    new_df = get_stock_df_from_csv("HDFC_BO")
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_ichimoku(new_df)
    new_df.to_csv(PATH + 'HDFC_BO' + '.csv')
except Exception as e:
    print(e)

Working on : HDFC_BO


In [26]:
# Plot the bollinger bands

def plot_with_bollinger_bands(df, ticker):
    fig = go.Figure()
    
    candle = go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')
    upper_line = go.Scatter(x=df['Date'], y=df['upper_band'], line=dict(color='rgba(250, 0, 0, 0.7)', width=1), name='Upper Band')
    lower_line = go.Scatter(x=df['Date'], y=df['lower_band'], line=dict(color='rgba(0, 250, 0, 0.7)', width=1), name='Lower Band')
    mid_line = go.Scatter(x=df['Date'], y=df['middle_band'], line=dict(color='rgba(0, 0, 250, 0.7)', width=1), name='Middle Band')
    
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(lower_line)
    fig.add_trace(mid_line)
    
    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    
    fig.update_layout(title=ticker + " Bollinger Bands", height=700, width=980, showlegend=True)
    fig.show()
    

In [28]:
test_df = get_stock_df_from_csv("HDFC_BO")
plot_with_bollinger_bands(test_df, "HDFC_BO")

# display(test_df)

In [41]:
# Plot the ichimoku

def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.3)'
    else:
        return 'rgba(250, 0, 0, 0.3)'
    

def plot_ichimoku(df, ticker):
    
    candle = go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df["Low"], close=df['Close'], name="Candlestick")

    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    df = df.groupby('group')

    dfs = []
    for name, data in df:
        dfs.append(data)

    for df in dfs:
        fig.add_traces(go.Scatter(x=df['Date'], y=df.SpanA, line=dict(color='rgba(0,0,0,0)')))
        fig.add_traces(go.Scatter(x=df['Date'], y=df.SpanB, line=dict(color='rgba(0,0,0,0)'), fill='tonexty', fillcolor=get_fill_color(df['label'].iloc[0])))

    baseline = go.Scatter(x=df['Date'], y=df1['Baseline'],line=dict(color='pink', width=2), name="Baseline")
    conversion = go.Scatter(x=df['Date'], y=df1['Conversion'],line=dict(color='black', width=1), name="Conversion")
    lagging = go.Scatter(x=df['Date'], y=df1['Lagging'],line=dict(color='purple', width=2), name="Lagging")
    span_a = go.Scatter(x=df['Date'], y=df1['SpanA'],line=dict(color='green', width=2, dash='dot'), name="Span A")
    span_b = go.Scatter(x=df['Date'], y=df1['SpanB'],line=dict(color='red', width=1, dash='dot'), name="Span B")

    fig.add_trace(candle)
#     fig.add_trace(baseline)
#     fig.add_trace(conversion)
#     fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    
    fig.update_layout(height=700, width=980, showlegend=True)
    fig.show()

In [42]:
# test_df = get_stock_df_from_csv("HDFC_BO")
plot_ichimoku(test_df, "HDFC_BO")